# Structured Logging Demo

In [2]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent.parent
sys.path.insert(0, str(PROJECT_ROOT))

In [3]:
import sqlite3
from importlib import import_module
from app.factories.logging_provider import LoggingProvider
from app.extensions.context_providers.dummy_context_provider import DummyContextProvider
from app.extensions.tool_providers.dummy_tool_provider import DummyToolProvider
from app.extensions.agents.dummy_agent import DummyAgent
from app.extensions.experiment_provider import ExperimentProvider

In [4]:
from app.utilities.db import init_db
import sqlite3
from app.factories.logging_provider import LoggingProvider
from app.extensions.context_providers import DummyContextProvider
from app.extensions.tool_providers import DummyToolProvider
from app.extensions.agents import DummyAgent
from app.extensions.experiment_provider import ExperimentProvider

# Connect to the in-memory database
conn = sqlite3.connect(":memory:")

# Initialize all required logging tables explicitly
init_db(conn)

# Pass the initialized connection to your logger
logger = LoggingProvider(connection=conn)

# Instantiate other components
ctx = DummyContextProvider(logger=logger)
tool = DummyToolProvider(logger=logger)
agent = DummyAgent(logger=logger)

# Now this will work without the OperationalError
exp = ExperimentProvider(
    "demo",
    "demo experiment",
    "test",
    "v1",
    1,
    0.0,
    "engine",
    "eval",
    "1.0",
    logger=logger,
)
ctx.get_context(experiment_id="demo", round=0)
tool.run(experiment_id="demo", round=0)
agent.run(experiment_id="demo", round=0)
exp.complete(1.0, True, "done")

In [5]:
cur = conn.cursor()
tables = [
    "context_retrieval_log",
    "tool_invocation_log",
    "agent_action_log",
    "system_event_log",
    "experiment_log",
]
counts = {t: cur.execute(f"SELECT COUNT(*) FROM {t}").fetchone()[0] for t in tables}
print(counts)

{'context_retrieval_log': 1, 'tool_invocation_log': 1, 'agent_action_log': 1, 'system_event_log': 0, 'experiment_log': 2}
